In [ ]:
import csv
import time
import random
import numpy as np
import networkx as nx
import strawberryfields as sf
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from strawberryfields import ops
from itertools import combinations
from strawberryfields.apps import data, sample, subgraph, plot, qchem
from strawberryfields.apps.similarity import feature_vector_orbits_sampling
from strawberryfields.utils import random_interferometer

Step1: Mapping

In [ ]:
# function that take a raw matrix and k (number of columns) and n (the nth number you want to compare) value in, return the best k columns
def find_best_k_combination(matrix, k, n):
    # get all possible combination
    num_columns = matrix.shape[1]
    column_combinations = combinations(range(num_columns), k)
        
#     column_combinations = get_column_combinations(matrix, k)
    
    strength = []
    Combination = []

    for combination in column_combinations:
        Combination.append(combination)
        selected_columns = matrix[:, combination]
        # calculate the row abs square sum
        row_sums = np.sum(np.abs(selected_columns) ** 2, axis=1)
        # make it decrease
        sorted_indices = np.argsort(row_sums)[::-1]
        sorted_row_sums = row_sums[sorted_indices]
        val = sorted_row_sums[n-1]
        strength.append(val)

    largest_value = np.max(strength)
    largest_index = np.argmax(strength)

    # find the best combination
    best_combination = Combination[largest_index]
    best_selected_columns = matrix[:, best_combination]

    return best_selected_columns

In [ ]:
# final function, recieve the original matrix and a list of len, also n (the nth number you want to compare), return the permuted matrix
# U = row_per_matrix*U'*col_per_matrix
def mapping(U, len_list, n):
    
    M, N = U.shape
    total_len = sum(len_list)
    assert N == total_len
    
    # copy U
    U_t = U.copy()
#     print(U)
    
    # new column permutation
    for i, length in enumerate(len_list):
        if i<len(len_list)-1:
            U_new = U_t[:,:length].copy()
#             print("i:", i)
            # calculate the row abs square sum
            row_sums = np.sum(np.abs(U_new) ** 2, axis=1)
            U_new_update = U_new.copy()
            # iterate remain column
            for j in range(np.sum(len_list[i+1:])):
#                 print("j:", j)
                for k in range(length):
#                     print("k:", k)
                    U_new_update_k = U_new_update[:,k].copy()
                    U_new_update[:,k] = U_t[:,length+j].copy()
                    # calculate the row abs square sum
                    row_sums_update = np.sum(np.abs(U_new_update) ** 2, axis=1)
                
                    if np.sort(row_sums_update)[::-1][n-1] > np.sort(row_sums)[::-1][n-1]:
#                     row_sums_update[n-1] > row_sums[n-1]:
                        U_new = U_new_update.copy()
                        U_t[:,length+j] = U_new_update_k.copy()
                        U_t[:,:length] = U_new.copy()
                        row_sums = row_sums_update
                    else:
                        U_new_update = U_new.copy()

            
            # add U_new to U_per
            if i==0:
                U_per = U_new.copy()
            elif i==len(len_list)-2:
                U_per = np.hstack((U_per, U_t)).copy()
            else:
                U_per = np.hstack((U_per, U_new)).copy()

            
            # drop U_new from U_t
            U_t = U_t[:,length:].copy()
        
                    

#     for i, length in enumerate(len_list):
#         U_new = find_best_k_combination(U_t, length, n)
        
#         # Find column indices in the larger matrix corresponding to the smaller matrix
#         selected_columns = []
#         for col in range(U_new.shape[1]):
#             for j in range(U_t.shape[1]):
#                 if np.array_equal(U_new[:, col], U_t[:, j]):
#                     selected_columns.append(j)
#                     break

#         # drop U_new from U_t
#         U_t = np.delete(U_t, selected_columns, axis=1)
        
#         # add U_new to U_per
#         if i==0:
#             U_per = U_new
#         else:
#             U_per = np.hstack((U_per, U_new))
      
      
    # find the permutation from U_per to U
    col_permutation = []
    for col in range(U_per.shape[1]):
        for i in range(U.shape[1]):
            if np.array_equal(U_per[:, col], U[:, i]):
                col_permutation.append(i)
                break
                
    # find the columns permutation matrix
    col_per_matrix = np.zeros((N, N))
    for i in range(N):
        col_per_matrix[col_permutation[i], i] = 1
        


    # calculate the sum of first len for U_per
    selected_columns = U_per[:, :len_list[0]] 
    # calculate the row abs square sum
    row_sums = np.sum(np.abs(selected_columns) ** 2, axis=1)
    
    # decide the row rotation
    sorted_indices = np.argsort(row_sums)
    U_per_new = U_per[sorted_indices]
    
    # Find row permutation
    row_permutation = []
    for row in range(U_per_new.shape[0]):
        for i in range(U_per.shape[0]):
            if np.array_equal(U_per_new[row, :], U_per[i, :]):
                row_permutation.append(i)
                break
                
    # find the rows permutation matrix
    row_per_matrix = np.zeros((M, M))
    for i in range(M):
        row_per_matrix[i, row_permutation[i]] = 1
        
    
    U_final = U_per_new
    
    # U_final = U_per, out put U_final and the permutation
    return U_final, col_per_matrix.T, row_per_matrix.T

Step2: Matrix Decomposition

In [ ]:
# Define the left eliminate equations
def left_equations(variables,parameters):
    theta, phi = variables
    a_r, a_i, b_r, b_i = parameters
    eq1 = a_r * np.cos(phi) * np.cos(theta) + a_i * np.sin(phi) * np.cos(theta) - b_r * np.sin(theta)
    eq2 = a_i * np.cos(phi) * np.cos(theta) - a_r * np.sin(phi) * np.cos(theta) - b_i * np.sin(theta)
    return [eq1, eq2]

# function of left elimination
def left_elimination(a,b):
    a_r = np.real(a)
    a_i = np.imag(a)
    b_r = np.real(b)
    b_i = np.imag(b)
    
    # Solve the system of equations
    initial_guess = [0, 0]  # Initial guess for the variables
    parameters = [a_r, a_i, b_r, b_i]
    raw_solution = fsolve(left_equations, initial_guess, args=(parameters,))
    
    
    if np.abs(a_r) < 1e-8 and np.abs(b_i) < 1e-8:
            raw_solution = [np.arctan(a_i/b_r), np.pi/2]
        
    if np.abs(a_i) < 1e-8 and np.abs(b_r) < 1e-8:
            raw_solution = [np.arctan(-a_r/b_i), np.pi/2]
            
            
    if np.abs(b_r) < 1e-8 and np.abs(b_i) < 1e-8:
            raw_solution = [np.pi/2, 0]
    
    
    normalized_solution = [angle % (2 * np.pi) for angle in raw_solution]
    solution = []
    for angle in normalized_solution:
        if angle < np.pi:
            solution.append(angle)
        else:
            solution.append(angle-2*np.pi)

    # Print the solution
#     print("Solution:", solution)
    return solution

In [ ]:
# Define the right eliminate equations
def right_equations(variables,parameters):
    theta, phi = variables
    a_r, a_i, b_r, b_i = parameters
    eq1 = a_r * np.cos(phi) * np.sin(theta) + a_i * np.sin(phi) * np.sin(theta) + b_r * np.cos(theta)
    eq2 = a_i * np.cos(phi) * np.sin(theta) - a_r * np.sin(phi) * np.sin(theta) + b_i * np.cos(theta)
    return [eq1, eq2]

# function of right elimination
def right_elimination(a,b):
    a_r = np.real(a)
    a_i = np.imag(a)
    b_r = np.real(b)
    b_i = np.imag(b)
    
    # Solve the system of equations
    initial_guess = [0, 0]  # Initial guess for the variables
    parameters = [a_r, a_i, b_r, b_i]
    raw_solution = fsolve(right_equations, initial_guess, args=(parameters,))
    
    if np.abs(a_r) < 1e-8 and np.abs(b_i) < 1e-8:
        raw_solution = [np.arctan(-b_r/a_i), np.pi/2]
        
    if np.abs(a_i) < 1e-8 and np.abs(b_r) < 1e-8:
        raw_solution = [np.arctan(b_i/a_r), np.pi/2]
        
    if np.abs(a_r) < 1e-8 and np.abs(a_i) < 1e-8:
        raw_solution = [np.pi/2, 0]
    
    normalized_solution = [angle % (2 * np.pi) for angle in raw_solution]
    solution = []
    for angle in normalized_solution:
        if angle < np.pi:
            solution.append(angle)
        else:
            solution.append(angle-2*np.pi)

    # Print the solution
#     print("Solution:", solution)
    return solution

In [ ]:
# give rotation matrix with idx_1 < idx_2
def rotation(theta, phi, n, idx_1, idx_2):
    I = np.eye(n, dtype=complex)
    I[idx_1,idx_1] = np.cos(theta)*(np.cos(phi)-np.sin(phi)*1j)
    I[idx_1,idx_2] = np.sin(theta)*(np.cos(phi)-np.sin(phi)*1j)
    I[idx_2,idx_1] = -np.sin(theta)
    I[idx_2,idx_2] = np.cos(theta)
    
    return I

In [ ]:
# matrix permutation
def permutation(U):
    
    rows, columns = U.shape
    assert(rows==columns)
    
    # create the permutation matrix
    Permutation = np.zeros([rows,columns])
    
    diag_U = np.eye(rows, dtype=complex)
    
    indices = np.nonzero(U)
    row_info = indices[0]
    col_info = indices[1]
    
    for i in range(rows):
        col = col_info[i]
        row = row_info[i]
        diag_U[col,:] = U[row,:]
        Permutation[col,row] = 1
        
    return diag_U, Permutation.T

In [ ]:
def phase(x):
    x_r = np.real(x)
    x_i = np.imag(x)
    
    raw_phi = np.arccos(x_r)
    if x_i > 0:
        t_phi = raw_phi
    else:
        t_phi = -raw_phi
        
    normalized_solution = t_phi % (2 * np.pi)
    if normalized_solution > np.pi:
        normalized_solution = normalized_solution-2*np.pi

    return normalized_solution

In [ ]:
# the all_list tell the decomposition order
# [[1,2], [2,3], [3,4]] means first use 2 eliminate 1, then use 3 eliminate 2, then use 4 eliminate 3
def U_decompose_plus(U, all_list):
    
    # pick up a machine precision
    threshold = 1e-8
    
    # get the shape of unitary
    rows, columns = U.shape
    assert(rows==columns)
    
    # create the rotation record matrix, the first row for theta, the second row for phi 
    # third row for low index, forth row for high index
    Theta = []
    Phi = []
    Low_idx = []
    High_idx = []
    
    # create the diagonal record matrix
    Diag = []
    

    for i in range(rows-1, 0, -1):
        list_i = all_list[i-1]
        row = U[i, :]
        
        for j in range(i):
            pair = list_i[j]
            
            
            # a is the number to be eliminated, b is the number to eliminate the previous one
            a = row[pair[0]]
            b = row[pair[1]]
            
            
#             if i % 3 != 0:
#                 if j == i-1:
#                     a = row[pair[1]]
#                     b = row[pair[0]]
                
                
            # choose the elimination method by position
            if pair[0] < pair[1]: 
                theta, phi = left_elimination(a,b)

                # write down the parameter
                Theta.append(theta)
                Phi.append(phi)
                Low_idx.append(pair[0])
                High_idx.append(pair[1])

                # create the rotation
                r = rotation(theta, phi, rows, pair[0], pair[1])
        

                # update Unitary
                U = np.dot(U, r)
                U= np.where(np.abs(U) < threshold, 0, U)

            else:
                theta, phi = right_elimination(b,a)

                # write down the parameter
                Theta.append(theta)
                Phi.append(phi)
                Low_idx.append(pair[1])
                High_idx.append(pair[0])

                # create the rotation
                r = rotation(theta, phi, rows, pair[1], pair[0])

                # update Unitary
                U = np.dot(U, r)
                U= np.where(np.abs(U) < threshold, 0, U)

            row = U[i, :]
            
#             if (np.count_nonzero(row) == 1):
            if j == i-1:
                entry = U[i, pair[1]]
                U[:, pair[1]] = 0
                U[i, :] = 0
                U[i, pair[1]] = entry
                flag = 0
    
    # do the permutation, and record
    U, Permutation = permutation(U)
    
    # find the phase shift
    for i in range(rows):
        x = U[i,i]/np.abs(U[i,i])
        phi_x = phase(x)
        Diag.append(phi_x)
        U[i,i] = U[i,i]*(np.cos(phi_x)-np.sin(phi_x)*1j)
    
    
    return U, Theta, Phi, Low_idx, High_idx, Diag, Permutation

Step3: gate drop

In [ ]:
# Step1 probability
def calculate_probability_sequence(angles, threshold, N):
    abs_angles = np.abs(angles)
    angles1 = abs_angles/(threshold)
    angles2 = angles1**N
    total_magnitude = np.sum(angles2)
    probabilities = angles2 / total_magnitude
    return probabilities

In [ ]:
# sequence is the origianl angel sequence, probability is calculate using calculate_probability_sequence(angles), percentage is the proportion you want to preserve
def pick_entries_with_indices(sequence, probabilities, proportion):
    
    N = len(sequence)
    num_entries = np.floor(N*proportion).astype(int)
    
    idx_sequence = np.arange(N).astype(int)
    
    if np.count_nonzero(probabilities) < num_entries:
        num_entries = np.count_nonzero(probabilities)
    
    picked_indices = np.random.choice(idx_sequence, size=num_entries, replace=False, p=probabilities)
    picked_entries = [sequence[index] for index in picked_indices]

    # Sort the picked entries and indices based on the indices
    picked_entries, picked_indices = zip(*sorted(zip(picked_entries, picked_indices), key=lambda x: x[1]))
    
    modified_sequence = [entry if index in picked_indices else 0 for index, entry in enumerate(sequence)]

    return modified_sequence

Step4: Matric Reconstruction

In [ ]:
def make_entries_zero(Array, threshold):
    array = Array.copy()
    for i in range(len(array)):
        if np.abs(array[i]) < threshold:
            array[i] = 0
    return array

In [ ]:
# give rotation matrix with idx_1 < idx_2
def reconstruct_rotation(theta, phi, n, idx_1, idx_2):
    I = np.eye(n, dtype=complex)
    I[idx_1,idx_1] = np.cos(theta)*(np.cos(phi)+np.sin(phi)*1j)
    I[idx_1,idx_2] = -np.sin(theta)
    I[idx_2,idx_1] = np.sin(theta)*(np.cos(phi)+np.sin(phi)*1j)
    I[idx_2,idx_2] = np.cos(theta)
    
    return I

In [ ]:
def matrix_reconstruct(Theta, Phi, Low_idx, High_idx, Diag, Permutation, N):
    
    M = len(Theta)
    
    per_theta = (len([x for x in Theta if np.abs(x) == 0]) / len(Theta)) * 100
#     print("theta reduce")
#     print(per_theta)
    
    per_phi = (len([x for x in Phi if np.abs(x) == 0]) / len(Phi)) * 100
#     print("phi reduce")
#     print(per_phi)

#     new_theta = Theta
#     new_phi = Phi
    
    V = np.eye(N, dtype=complex)
    
    # rotation
    for i in range(M):
        r = reconstruct_rotation(Theta[i], Phi[i], N, Low_idx[i], High_idx[i])
        V = np.dot(r, V)
        
    # phase shift
    for i in range(N):
        V[i,:] = V[i,:]*(np.cos(Diag[i])+np.sin(Diag[i])*1j)
        
    # permutation
    V = np.dot(Permutation, V)
    
    return V

Accuracy

In [ ]:
# function to calculate the approximation accuracy
def accuracy(U,U_app):
    I = np.dot(U,np.conjugate(U_app).transpose())
    N, N = I.shape
    acc = np.trace(I)/N
    return acc

In [ ]:
def unitary_accuracy(M, U1, dec_list, maps, len_list, n, proportion1, th, N):
    
    N, N = U1.shape
    
    # mapping
    if maps:
        U1_map, col_per_matrix, row_per_matrix = mapping(U1, len_list, n)
    else:
        U1_map = U1
        
    # get the decomposition information of U1
    _, Theta1, Phi1, Low_idx1, High_idx1, Diag1, Permutation1 = U_decompose_plus(U1_map, dec_list)
    
    # modify Theta
    Theta_prob1 = calculate_probability_sequence(Theta1, th, N)
    
    k = 0
    acc1 = []
    while k < M:

        # get the new Theta from the probability distribution
        new_Theta1 = pick_entries_with_indices(Theta1, Theta_prob1, proportion1)
        new_Phi1 = Phi1
        
        # reconstruction of U2 using new angel
        U1_map_app = matrix_reconstruct(new_Theta1, new_Phi1, Low_idx1, High_idx1, Diag1, Permutation1, N)
        # approximation accuracy
        acc1_t = accuracy(U1_map,U1_map_app)
        acc1.append(acc1_t)
        
        k = k+1
        
    acc1_mean = np.mean(acc1)
    print("mean accuracy for U1:", acc1_mean)    

Reduce counting

In [ ]:
def count_numbers_less_than(numbers, threshold):
    count = 0
    for number in numbers:
        if np.abs(number) < threshold:
            count += 1
    return count

In [ ]:
def generate_random_graph(nodes, probability):
    G = nx.Graph()
    G.add_nodes_from(nodes)

    for i in nodes:
        for j in nodes:
            if i != j and random.random() < probability:
                G.add_edge(i, j)

    return G

Code for List

In [ ]:
List = [[] for _ in range(500)]

# List[i] = list_orderi
for i in range(499, -1, -1):
    for j in range(1, 500, 3):
        List[i].append([j-1,j])
        List[i].append([j+1,j])
        List[i].append([j,j+3])
        
    
    List[i] = List[i][:i+1]
    
    
    if i%3 != 2:
        last_item = List[i][-1]
        last_item[0], last_item[1] = last_item[1], last_item[0]
        List[i][-1] = last_item


In [ ]:
dec_time = []
total_time = []
BS_reduce = []
fidelity = []

500 scale

In [ ]:
List_500 = List[:499]

In [ ]:
len_list_500 = [167]
for i in range(333):
    len_list_500.append(1)

In [ ]:
# Loading the CSV file back into a numpy array
U = np.loadtxt('500_U5.csv', delimiter=',', dtype=complex)

In [ ]:
start_time = time.time()
U_map, col_per_matrix, row_per_matrix = mapping(U ,len_list_500, 167)
end_time = time.time()
execution_time1 = end_time - start_time
start_time = time.time()
_, Theta, Phi, Low_idx, High_idx, Diag, Permutation = U_decompose_plus(U_map, List_500)
end_time = time.time()
execution_time2 = end_time - start_time
print(execution_time2)

In [ ]:
start_time = time.time()
new_Theta = make_entries_zero(Theta, 0.045)
U_app = matrix_reconstruct(new_Theta, Phi, Low_idx, High_idx, Diag, Permutation, 500)
end_time = time.time()
execution_time3 = end_time - start_time
# print(np.count_nonzero(new_Theta))
gate_r = (1-np.count_nonzero(new_Theta)/len(Theta))
f = accuracy(U_map,U_app)

In [ ]:
execution_time = execution_time1+execution_time2+execution_time3
print(execution_time)

In [ ]:
BS_reduce.append(gate_r)
fidelity.append(f)
dec_time.append(execution_time2)
total_time.append(execution_time)

200 scale

In [ ]:
List_200 = List[:199]

In [ ]:
len_list_200 = [67]
for i in range(133):
    len_list_200.append(1)

In [ ]:
# Loading the CSV file back into a numpy array
U = np.loadtxt('200_U3.csv', delimiter=',', dtype=complex)

In [ ]:
start_time = time.time()
U_map, col_per_matrix, row_per_matrix = mapping(U ,len_list_200, 67)
end_time = time.time()
execution_time1 = end_time - start_time
start_time = time.time()
_, Theta, Phi, Low_idx, High_idx, Diag, Permutation = U_decompose_plus(U_map, List_200)
end_time = time.time()
execution_time2 = end_time - start_time
print(execution_time2)

In [ ]:
start_time = time.time()
new_Theta = make_entries_zero(Theta, 0.072)
U_app = matrix_reconstruct(new_Theta, Phi, Low_idx, High_idx, Diag, Permutation, 200)
end_time = time.time()
execution_time3 = end_time - start_time
# print(np.count_nonzero(new_Theta))
gate_r = (1-np.count_nonzero(new_Theta)/len(Theta))
f = accuracy(U_map,U_app)

In [ ]:
execution_time = execution_time1+execution_time2+execution_time3
print(execution_time)

In [ ]:
BS_reduce.append(gate_r)
fidelity.append(f)
dec_time.append(execution_time2)
total_time.append(execution_time)

100 scale

In [ ]:
List_100 = List[:99]

In [ ]:
List_100[-1][-1][-1]=99

In [ ]:
len_list_100 = [34]
for i in range(66):
    len_list_100.append(1)

In [ ]:
# Loading the CSV file back into a numpy array
U = np.loadtxt('100_U1.csv', delimiter=',', dtype=complex)

In [ ]:
start_time = time.time()
U_map, col_per_matrix, row_per_matrix = mapping(U ,len_list_100, 34)
end_time = time.time()
execution_time1 = end_time - start_time
start_time = time.time()
_, Theta, Phi, Low_idx, High_idx, Diag, Permutation = U_decompose_plus(U_map, List_100)
end_time = time.time()
execution_time2 = end_time - start_time
print(execution_time2)

In [ ]:
start_time = time.time()
new_Theta = make_entries_zero(Theta, 0.105)
U_app = matrix_reconstruct(new_Theta, Phi, Low_idx, High_idx, Diag, Permutation, 100)
end_time = time.time()
execution_time3 = end_time - start_time
# print(np.count_nonzero(new_Theta))
gate_r = (1-np.count_nonzero(new_Theta)/len(Theta))
f = accuracy(U_map,U_app)

In [ ]:
execution_time = execution_time1+execution_time2+execution_time3
print(execution_time)

In [ ]:
BS_reduce.append(gate_r)
fidelity.append(f)
dec_time.append(execution_time2)
total_time.append(execution_time)

In [ ]:
print(BS_reduce)

60 scale

In [ ]:
List_60 = List[:59]

In [ ]:
len_list_60 = [20]
for i in range(40):
    len_list_60.append(1)

In [ ]:
# Loading the CSV file back into a numpy array
U = np.loadtxt('60_U1.csv', delimiter=',', dtype=complex)

In [ ]:
start_time = time.time()
U_map, col_per_matrix, row_per_matrix = mapping(U ,len_list_60, 20)
end_time = time.time()
execution_time1 = end_time - start_time
start_time = time.time()
_, Theta, Phi, Low_idx, High_idx, Diag, Permutation = U_decompose_plus(U_map, List_60)
end_time = time.time()
execution_time2 = end_time - start_time
print(execution_time2)

In [ ]:
start_time = time.time()
new_Theta = make_entries_zero(Theta, 0.143)
U_app = matrix_reconstruct(new_Theta, Phi, Low_idx, High_idx, Diag, Permutation, 60)
end_time = time.time()
execution_time3 = end_time - start_time
# print(np.count_nonzero(new_Theta))
gate_r = (1-np.count_nonzero(new_Theta)/len(Theta))
f = accuracy(U_map,U_app)

In [ ]:
execution_time = execution_time1+execution_time2+execution_time3
print(execution_time)

In [ ]:
BS_reduce.append(gate_r)
fidelity.append(f)
dec_time.append(execution_time2)
total_time.append(execution_time)

20 scale

In [ ]:
List_20 = List[:19]

In [ ]:
len_list_20 = [7]
for i in range(13):
    len_list_20.append(1)

In [ ]:
# Loading the CSV file back into a numpy array
U = np.loadtxt('20_U1.csv', delimiter=',', dtype=complex)

In [ ]:
start_time = time.time()
U_map, col_per_matrix, row_per_matrix = mapping(U ,len_list_20, 7)
end_time = time.time()
execution_time1 = end_time - start_time
start_time = time.time()
_, Theta, Phi, Low_idx, High_idx, Diag, Permutation = U_decompose_plus(U_map, List_20)
end_time = time.time()
execution_time2 = end_time - start_time
print(execution_time2)

In [ ]:
start_time = time.time()
new_Theta = make_entries_zero(Theta, 0.25)
U_app = matrix_reconstruct(new_Theta, Phi, Low_idx, High_idx, Diag, Permutation, 20)
end_time = time.time()
execution_time3 = end_time - start_time
# print(np.count_nonzero(new_Theta))
gate_r = (1-np.count_nonzero(new_Theta)/len(Theta))
f = accuracy(U_map,U_app)

In [ ]:
execution_time = execution_time1+execution_time2+execution_time3
print(execution_time)

In [ ]:
BS_reduce.append(gate_r)
fidelity.append(f)
dec_time.append(execution_time2)
total_time.append(execution_time)

15 scale

In [ ]:
List_15 = List[:14]

In [ ]:
len_list_15 = [5]
for i in range(10):
    len_list_15.append(1)

In [ ]:
# Loading the CSV file back into a numpy array
U = np.loadtxt('15_U2.csv', delimiter=',', dtype=complex)

In [ ]:
start_time = time.time()
U_map, col_per_matrix, row_per_matrix = mapping(U ,len_list_15, 5)
end_time = time.time()
execution_time1 = end_time - start_time
start_time = time.time()
_, Theta, Phi, Low_idx, High_idx, Diag, Permutation = U_decompose_plus(U_map, List_15)
end_time = time.time()
execution_time2 = end_time - start_time
print(execution_time2)

In [ ]:
start_time = time.time()
new_Theta = make_entries_zero(Theta, 0.405)
U_app = matrix_reconstruct(new_Theta, Phi, Low_idx, High_idx, Diag, Permutation, 15)
end_time = time.time()
execution_time3 = end_time - start_time
# print(np.count_nonzero(new_Theta))
gate_r = (1-np.count_nonzero(new_Theta)/len(Theta))
f = accuracy(U_map,U_app)

In [ ]:
execution_time = execution_time1+execution_time2+execution_time3
print(execution_time)

In [ ]:
BS_reduce.append(gate_r)
fidelity.append(f)
dec_time.append(execution_time2)
total_time.append(execution_time)

10 scale

In [ ]:
List_10 = List[:9]

In [ ]:
List_10[-1][-1][-1]=9

In [ ]:
len_list_10 = [4]
for i in range(6):
    len_list_10.append(1)

In [ ]:
# Loading the CSV file back into a numpy array
U = np.loadtxt('10_U1.csv', delimiter=',', dtype=complex)

In [ ]:
start_time = time.time()
U_map, col_per_matrix, row_per_matrix = mapping(U ,len_list_10, 4)
end_time = time.time()
execution_time1 = end_time - start_time
start_time = time.time()
_, Theta, Phi, Low_idx, High_idx, Diag, Permutation = U_decompose_plus(U_map, List_10)
end_time = time.time()
execution_time2 = end_time - start_time
print(execution_time2)

In [ ]:
start_time = time.time()
new_Theta = make_entries_zero(Theta, 0.65)
U_app = matrix_reconstruct(new_Theta, Phi, Low_idx, High_idx, Diag, Permutation, 10)
end_time = time.time()
execution_time3 = end_time - start_time
# print(np.count_nonzero(new_Theta))
gate_r = (1-np.count_nonzero(new_Theta)/len(Theta))
f = accuracy(U_map,U_app)

In [ ]:
execution_time = execution_time1+execution_time2+execution_time3
print(execution_time)

In [ ]:
BS_reduce.append(gate_r)
fidelity.append(f)
dec_time.append(execution_time2)
total_time.append(execution_time)

In [ ]:
print(total_time)

In [ ]:
scale = [500, 200, 100, 60, 20, 15, 10]

In [ ]:
scale.insert(0, 'Qumode#')
BS_reduce.insert(0, 'BS Gate# drop')
dec_time.insert(0, 'Decomp time')
total_time.insert(0, 'Total time')
fidelity.insert(0, 'Fidelity')

In [ ]:
result = [list(row) for row in zip(scale, BS_reduce, dec_time, total_time, fidelity)]

In [ ]:
# Specify the file name
file_name = 'Table 3.csv'

# Write the combined list to a CSV file
with open(file_name, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(result)